When there is no data file for the molecule you want, or no database to get it from, you have to define your atoms geometry by hand. Here is how that is done for a CO molecule (Figure 1). We must define the type and position of each atom, and the unit cell the atoms are in.

In [29]:
from ase import Atoms, Atom
from ase.io import write
atoms = Atoms(
    [Atom('C', [0,0,0]),
    Atom('O', [1.1,0,0]),],
    cell = (10,10,10)
)

write("./images/simple-cubic-cell.png", atoms, rotation='50z,-80x') # "ase info --formats" in command line shows all formats

There are two inconvenient features of the simple cubic cell: 1. Since the CO molecule is at the corner, its electron density is spread over the 8 corners of the box, which is not convenient for visualization later (see Visualizing electron density). 2. Due to the geometry of the cube, you need fairly large cubes to make sure the electron density of the molecule does not overlap with that of its images. Electron-electron interactions are repulsive, and the overlap makes the energy increase significantly. Here, the CO molecule has 6 images due to periodic boundary conditions that are 10 Å away. The volume of the unit cell is 1000 Å3. The first problem is easily solved by centering the atoms in the unit cell. The second problem can be solved by using a face-centered cubic lattice, which is the lattice with the closest packing. We show the results of the centering in Figure 2, where we have guessed values for b until the CO molecules are on average 10 Å apart. Note the final volume is only about 715 Å3, which is smaller than the cube. This will result in less computational time to compute properties.

In [30]:
from ase import Atoms, Atom
from ase.io import write
from ase.visualize import view
b=7.1

# fcc 原胞
atoms = Atoms(
    [Atom('C', [0,0,0]),
    Atom('O', [1.1,0,0]),],
    cell = [
        [b,b,0],
        [b,0,b],
        [0,b,b]
    ]
)
atoms.center() # translate atoms to center of unit cell
# view(atoms)
write("./images/fcc-cell.png", atoms, show_unit_cell=2) # In the write command we use the option show_unit_cell =2 to draw the unit cell boundaries.

At this point you might ask, "How do you know the distance to the neighboring image?" The ag viewer lets you compute this graphically, but we can use code to determine this too. All we have to do is figure out the length of each lattice vector, because these are what separate the atoms in the images. We use the numpy module to compute the distance of a vector as the square root of the sum of squared elements.

In [33]:
from ase import Atoms, Atom
from ase.io import write
import numpy as np
b=7.1

# fcc 原胞
atoms = Atoms(
    [Atom('C', [0,0,0]),
    Atom('O', [1.1,0,0]),],
    cell = [
        [b,b,0],
        [b,0,b],
        [0,b,b]
    ]
)
atoms.center() # translate atoms to center of unit cell
(a1, a2, a3) = atoms.get_cell()
print("|a1| = {0:1.2f} Ang".format(np.sum(a1**2)**0.5))
print("|a2| = {0:1.2f} Ang".format(np.linalg.norm(a2)))
print("|a3| = {0:1.2f} Ang".format(np.sum(a3**2)**0.5))

|a1| = 10.04 Ang
|a2| = 10.04 Ang
|a3| = 10.04 Ang


The first line is the number of atoms in the file. The second line is often a comment. What follows is one line per atom with the symbol and Cartesian coordinates in Å. Note that the XYZ format does not have unit cell information in it, so you will have to figure out a way to provide it. In this example, we center the atoms in a box with vacuum on all sides

In [3]:
from ase.io import write, read
from ase.visualize import view

atoms = read("molecules/isobutane.xyz")
atoms.center(vacuum=5) # 原本无真空层，添加真空层，由分子转为晶体
# view(atoms)
write("cell/isobutane.vasp", atoms,format="vasp")

ase defines a number of molecular geometries in the ase.data.molecules database.  
For example, the database includes the molecules in the G2/97 database  

In [18]:
from ase.data import g2
from ase.data import g2_1
import ase

print(dir(ase.data)[11:])
keys = g2.data.keys()
keys1 = g2_1.data.keys()
print(keys)

['atomic_masses', 'atomic_masses_common', 'atomic_masses_iupac2016', 'atomic_masses_legacy', 'atomic_names', 'atomic_numbers', 'chemical_symbols', 'colors', 'covalent_radii', 'g2', 'g2_1', 'g2_2', 'ground_state_magnetic_moments', 'missing', 'np', 'reference_states', 'symbol', 'vdw', 'vdw_radii']
dict_keys(['Be', 'BeH', 'C', 'C2H2', 'C2H4', 'C2H6', 'CH', 'CH2_s1A1d', 'CH2_s3B1d', 'CH3', 'CH3Cl', 'CH3OH', 'CH3SH', 'CH4', 'CN', 'CO', 'CO2', 'CS', 'Cl', 'Cl2', 'ClF', 'ClO', 'F', 'F2', 'H', 'H2CO', 'H2O', 'H2O2', 'HCN', 'HCO', 'HCl', 'HF', 'HOCl', 'Li', 'Li2', 'LiF', 'LiH', 'N', 'N2', 'N2H4', 'NH', 'NH2', 'NH3', 'NO', 'Na', 'Na2', 'NaCl', 'O', 'O2', 'OH', 'P', 'P2', 'PH2', 'PH3', 'S', 'S2', 'SH2', 'SO', 'SO2', 'Si', 'Si2', 'Si2H6', 'SiH2_s1A1d', 'SiH2_s3B1d', 'SiH3', 'SiH4', 'SiO', '2-butyne', 'Al', 'AlCl3', 'AlF3', 'B', 'BCl3', 'BF3', 'C2Cl4', 'C2F4', 'C2H3', 'C2H5', 'C2H6CHOH', 'C2H6NH', 'C2H6SO', 'C3H4_C2v', 'C3H4_C3v', 'C3H4_D2d', 'C3H6_Cs', 'C3H6_D3h', 'C3H7', 'C3H7Cl', 'C3H8', 'C3H9C'

Here is an example of getting the geometry of an acetonitrile molecule and writing an image to a file

In [4]:
from ase.structure import molecule
from ase.visualize import view
from ase.io import write

atoms = molecule("CH3CN")
# view(atoms)
atoms.center(vacuum=6)
# view(atoms)
write("images/ch3cu.png", atoms, show_unit_cell=2)


In [5]:
# rotated
from ase.structure import molecule
from ase.visualize import view
from ase.io import write

atoms = molecule("CH3CN")
atoms.center(vacuum=6)
write("images/ch3cu-rotated.png", atoms, show_unit_cell=2, rotation="45x, 45y")

If you actually want to rotate the coordinates, there is a nice way to do that too, with the ase.Atoms. rotate method. Actually there are some subtleties in rotation. One rotates the molecule an angle (in radians) around a vector, but you have to choose whether the center of mass should be fixed or not. You also must decide whether or not the unit cell should be rotated. In the next example you can see the coordinates have changed due to the rotations. Note that the write function uses the rotation angle in degrees, while the rotate function uses radians

In [30]:
from ase.structure import molecule
from ase.visualize import view
from ase.io import write
from ase.io.trajectory import Trajectory
from numpy import pi

traj = Trajectory("trajectorys/ch3cu-rotated-2.traj", "w")

atoms = molecule("CH3CN")
atoms.center(vacuum=6)

traj.write(atoms)

# center: The center is kept fixed under the rotation. Use 'COM' to fix the center of mass, 'COP' to fix the center of positions or 'COU' to fix the center of cell.
atoms.rotate(90, "x", center="COP", rotate_cell=False)
traj.write(atoms)

atoms.rotate(90, "y", center="COP", rotate_cell=False)
traj.write(atoms)

Combining Atoms objects

In [35]:
from ase.structure import molecule
from ase.visualize import view
from ase.io import write

atoms1 = molecule("CH3CN")
atoms2 = molecule("O2")
atoms2.translate([3, 0, 0]) # move along x axis with 3A

bothatoms = atoms1 + atoms2
bothatoms.center(5)
view(bothatoms)

Simple properties

In [62]:
from ase.structure import molecule
from ase.io import write

atoms = molecule("CH3CN")
# atoms.center(3)

print('#  sym     p_x     p_y     p_z')
for i, atom in enumerate(atoms):
    print("{0:3d}{1:^4s}{2:-8.2f}{3:-8.2f}{4:-8.2f}".format(i,atom.symbol,atom.x, atom.y,atom.z))

sym = atoms.get_chemical_symbols()
num = atoms.get_atomic_numbers()
pos = atoms.get_positions()
atom_indices = range(len(atoms))

print()
print("# sym    at#    p_x    p_y    p_z")
print("---------------------------------")
for i,s,n,p in zip(atom_indices, sym, num, pos):
    px,py,pz=p
    print("{:3d}{:>3s}{:4d}{:8.2f}{:8.2f}{:8.2f}".format(i, s, n, px, py, pz))


#  sym     p_x     p_y     p_z
  0 C      0.00    0.00   -1.19
  1 C      0.00    0.00    0.27
  2 N      0.00    0.00    1.45
  3 H      0.00    1.02   -1.56
  4 H      0.89   -0.51   -1.56
  5 H     -0.89   -0.51   -1.56

# sym    at#    p_x    p_y    p_z
---------------------------------
  0  C   6    0.00    0.00   -1.19
  1  C   6    0.00    0.00    0.27
  2  N   7    0.00    0.00    1.45
  3  H   1    0.00    1.02   -1.56
  4  H   1    0.89   -0.51   -1.56
  5  H   1   -0.89   -0.51   -1.56


Center of mass

In [12]:
from ase.structure import molecule
import numpy as np

atoms = molecule("NH3")
print(f"COM-center of mass : {atoms.get_center_of_mass()}")

pos = atoms.positions
masses = atoms.get_masses() # array([14.007,  1.008,  1.008,  1.008])
COM = np.array([00,0,0], dtype="f4")

for m, p in zip(masses, pos):
    COM += m*p
COM /= masses.sum()
COM

COM-center of mass : [0.00000000e+00 5.91861899e-08 4.75435401e-02]


array([-7.8016843e-11,  6.0165981e-08,  4.7543544e-02], dtype=float32)

The moment of inertia is a measure of resistance to changes in rotation. It is defined by I = ∑N i=1 mir2 i where ri is the distance to an axis of rotation. There are typically three moments of inertia, although some may be zero depending on symmetry, and others may be degenerate. There is a convenient function to get the moments of inertia: ase.Atoms.get_moments_of_inertia. Here are several examples of molecules with different types of symmetry.

In [14]:
from ase.structure import molecule
from ase.visualize import view
atoms = molecule("NH3")
atoms.get_moments_of_inertia()
# view(atoms)

If you want to know the principle axes of rotation, we simply pass vectors=True to the function, and it returns the moments of inertia and the principle axes

In [16]:
from ase.structure import molecule
from ase.visualize import view
atoms = molecule("CH3Cl")
# view(atoms)
inertia, axes = atoms.get_moments_of_inertia(vectors=True)
inertia, axes

(array([ 3.2039126 , 37.969823  , 37.96982492]),
 array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]]))

Computing bond lengths and angles
To calculate the distance between two atoms, you have to specify their indices, remembering that the index starts at 0.

In [25]:
from ase.structure import molecule
from ase.visualize import view
atoms = molecule("NH3")
print("atoms symbols")
print("=============")
for i,atom in enumerate(atoms):
    symbol = atom.symbol
    print(f"{i} {symbol}")
print("=============")
    
# distance between N and H
print(f"N to H1: {atoms.get_distance(0,1)}")
print(f"N to H2: {atoms.get_distance(0,2)}")
print(f"N to H3: {atoms.get_distance(0,3)}")
print(f"H1 to H2: {atoms.get_distance(1,2)}")
print("=============")

atoms symbols
0 N
1 H
2 H
3 H
N to H1: 1.0167934463645996
N to H2: 1.0167932803647945
N to H3: 1.0167932803647945
H1 to H2: 1.6276614450729612


Dihedral angles

In [4]:
from ase.structure import molecule
from ase.visualize import view
atoms = molecule("NH3")
# 0 指的是N，1，2，3为H
print(atoms.get_angle(1,0,2))
print(atoms.get_angle(1,0,3))
print(atoms.get_angle(2,0,3))

106.33462423179175
106.33462423179175
106.33468888231188
